In [1]:
from pathlib import Path
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from data_scripts import *

tex_fonts = {
    # Use LaTeX to write all text
    # "text.usetex": True,
    "font.family": "serif",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 10,
    "font.size": 10,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
}

sns.set_theme(
    # context="talk",
    context="paper",
    # rc={"figure.figsize": (8, 6)},
    palette="colorblind6",
    style="whitegrid",
    rc=tex_fonts,
)

CONVERGED = False
NUM_SIMS = 5
YEAR = 365 * 24 * 60 * 60
USE_SIMULATED_TIME = False

if USE_SIMULATED_TIME:
    X_LABEL = "Simulated time, years"
else:
    X_LABEL = "Time step number"


data = {
    "Only CPR": load_data('../1/thermal_cpr', n_newest=NUM_SIMS),
    "Only Schur": load_data('../1/thermal_schur', n_newest=NUM_SIMS),
    "CPR and Schur only": load_data('../1/thermal_dynamic', n_newest=NUM_SIMS),
    "Many solvers": load_data("thermal_many_solvers", n_newest=NUM_SIMS),
    "Many solvers gp": load_data("thermal_many_solvers_gp", n_newest=NUM_SIMS),
    # "Random": load_data("thermal_random", n_newest=NUM_SIMS),
    # "Source location coldstart": load_data("thermal_coldstart_source_location", n_newest=NUM_SIMS),
    # "Source location warmstart": load_data("thermal_warmstart_source_location", n_newest=NUM_SIMS),
}

save_media_path = Path("media")
save_media_path.mkdir(exist_ok=True)

Loading data:
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_cpr_38.npy
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_cpr_39.npy
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_cpr_40.npy
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_cpr_41.npy
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_cpr_42.npy


/home/firedrake/porepy/src/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


Loading data:
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_schur_31.npy
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_schur_32.npy
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_schur_33.npy
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_schur_34.npy
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_schur_35.npy
Loading data:
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_dynamic_45.npy
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_dynamic_46.npy
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_dynamic_47.npy
/home/firedrake/solver_selector/examples/4_extended_solver_space/../1/performance/thermal_dynamic_48.npy
/home/firedrake/solver_selector/examp

Can we choose 2-3 best solvers from the bigger list of solvers?
evaluate cumulative performance prediction of pairs (or n solvers) after the simulation. 

In [2]:
import itertools
from solver_selector.solver_selector import make_solver_selector
from thermal_solvers import make_thermal_solver_space

solver_space = make_thermal_solver_space("full")
all_solvers = solver_space.get_all_solvers()

def find_best_pairs(sim_data: Sequence[SolverSelectionData], n_best=1):
    solver_selector = make_solver_selector(
        solver_space,
        params={
            "exploration": 0,
            # 'load_statistics': sim_data,
            'load_statistics': data['Many solvers gp'][-3],
        },
    )

    solver_scores = {}

    for i, performance_predictor in enumerate(solver_selector.predictors):
        print(i)
        assert performance_predictor.is_initialized
        scores = []
        for entry in sim_data:
            solver = performance_predictor.select_solver_parameters(
                entry.prediction.context
            )
            scores.append(solver.score)
        solver_scores[i] = np.array(scores)

    new_combinations_scores = {}

    for comb in itertools.combinations(range(len(all_solvers)), 2):
        s0_scores = solver_scores[comb[0]]
        s1_scores = solver_scores[comb[1]]
        new_combinations_scores[comb] = np.maximum(s0_scores, s1_scores)

    comb_total = {comb: sum(score) for comb, score in new_combinations_scores.items()}
    return sorted(comb_total, key=comb_total.get, reverse=True)[:n_best]

In [3]:
solver_space = make_thermal_solver_space("full")
all_solvers = solver_space.get_all_solvers()

results = []

for sim_data in data['Many solvers']:
    best_solvers = find_best_pairs(sim_data, n_best=1)
    results.append(best_solvers)

for best_solvers in results:
    for i, comb in enumerate(best_solvers):
        print(i)
        for id in comb:
            default = all_solvers[id].use_defaults()
            conf = solver_space.config_from_decision(decision=default, optimized_only=True)
            print(solver_space.format_config(conf))
        print()

Selecting from 19 solvers.
0 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - gmres [amg, restart=10], method=full]
1 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - gmres [amg, restart=10], method=upper]
2 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - gmres [amg, restart=10], method=lower]
3 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - amg, method=full]
4 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - amg, method=upper]
5 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - amg, method=lower]
6 fgmres - schur_cd [primary - amg, secondary - gmres [amg, restart=10], method=full]
7 fgmres - schur_cd [primary - amg, secondary - gmres [amg, restart=10], method=upper]
8 fgmres - schur_cd [primary - amg, secondary - gmres [amg, restart=10], method=lower]
9 fgmres - schur_cd [primary - amg, secondary - amg, method=full]
10 fgmres - schur_cd [primary - amg, secondary - amg, method=upper]

In [4]:
results = []

for sim_data in data['Many solvers gp'][:1]:
    best_solvers = find_best_pairs(sim_data, n_best=5)
    results.append(best_solvers)

for best_solvers in results:
    for i, comb in enumerate(best_solvers):
        print(i)
        for id in comb:
            default = all_solvers[id].use_defaults()
            conf = solver_space.config_from_decision(decision=default, optimized_only=True)
            print(solver_space.format_config(conf))
        print()

Selecting from 19 solvers.
0 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - gmres [amg, restart=10], method=full]
1 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - gmres [amg, restart=10], method=upper]
2 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - gmres [amg, restart=10], method=lower]
3 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - amg, method=full]
4 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - amg, method=upper]
5 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - amg, method=lower]
6 fgmres - schur_cd [primary - amg, secondary - gmres [amg, restart=10], method=full]
7 fgmres - schur_cd [primary - amg, secondary - gmres [amg, restart=10], method=upper]
8 fgmres - schur_cd [primary - amg, secondary - gmres [amg, restart=10], method=lower]
9 fgmres - schur_cd [primary - amg, secondary - amg, method=full]
10 fgmres - schur_cd [primary - amg, secondary - amg, method=upper]

In [5]:
def find_best_solver(sim_data: Sequence[SolverSelectionData], n_best=5):
    solver_selector = make_solver_selector(
        solver_space,
        params={
            "exploration": 0,
            'load_statistics': sim_data,
        },
    )

    solver_scores = {}

    for i, performance_predictor in enumerate(solver_selector.predictors):
        print(i)
        assert performance_predictor.is_initialized
        scores = []
        for entry in sim_data:
            solver = performance_predictor.select_solver_parameters(
                entry.prediction.context
            )
            scores.append(solver.score)
        solver_scores[i] = np.array(scores)

    solver_total = {solv: sum(score) for solv, score in solver_scores.items()}
    return sorted(solver_total, key=solver_total.get, reverse=True)[:n_best]

In [6]:
from contextlib import contextmanager
import sys
import os

@contextmanager
def stdout_redirected():
    save_stdout = sys.stdout
    with open(os.devnull, 'w') as f:
        sys.stdout = f
        try:
            yield None
        finally:
            sys.stdout = save_stdout

In [7]:
from data_scripts import make_num_linear_iters
from tqdm import tqdm
results = []



for sim_data in tqdm(data['Many solvers gp']):
    with stdout_redirected():
        best_solvers = find_best_solver(sim_data, n_best=10)
        results.append(best_solvers)

for best_solvers in results:
    for i, comb in enumerate(best_solvers):
        default = all_solvers[i].use_defaults()
        conf = solver_space.config_from_decision(decision=default, optimized_only=True)
        print(i, solver_space.format_config(conf))
    print()


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:14<00:00,  2.81s/it]

0 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - gmres [amg, restart=10], method=full]
1 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - gmres [amg, restart=10], method=upper]
2 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - gmres [amg, restart=10], method=lower]
3 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - amg, method=full]
4 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - amg, method=upper]
5 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - amg, method=lower]
6 fgmres - schur_cd [primary - amg, secondary - gmres [amg, restart=10], method=full]
7 fgmres - schur_cd [primary - amg, secondary - gmres [amg, restart=10], method=upper]
8 fgmres - schur_cd [primary - amg, secondary - gmres [amg, restart=10], method=lower]
9 fgmres - schur_cd [primary - amg, secondary - amg, method=full]

0 fgmres - schur_cd [primary - gmres [amg, restart=10], secondary - gmres [amg, restart=10], 